# Café ou acarajé environment

In [17]:
import numpy as np

class CafeAcarajeGame:
    def __init__(self):
        self.states = ["F", "E", "D", "Café", "B", "Acarajé"]
        self.dice = {
            "D1": [1, 2, 3, 4, 5, 6],
            "D2": [1, 1, 2, 2, 3, 3],
            "D3": [1, 1, 1, 2, 2, 2]
        }
        self.state = "F"  # Estado inicial
    
    def roll_dice(self, dice_choice):
        if dice_choice not in self.dice:
            raise ValueError("Invalid dice choice")
        return np.random.choice(self.dice[dice_choice])
    
    def step(self, action):
        dice_choice = action
        roll = self.roll_dice(dice_choice)
        
        # Encontrar o índice do estado atual
        current_index = self.states.index(self.state)
        
        # Calcular o novo índice baseado no roll
        new_index = (current_index + roll) % len(self.states)
        
        # Atualizar o estado
        self.state = self.states[new_index]
        
        # Calcular a recompensa
        # reward = 1 if self.state == "Café" else 0
        done = self.state == "Café"
        reward = self.reward(self.state)
        
        return self.state, reward, done

    def reward(self, state):
        # Calcular a recompensa
        reward = 1 if state == "Café" else 0
        return reward
    
    def reset(self):
        self.state = "F"  # Estado inicial
        return self.state

    def transition_probability(self, state, action, next_state):
        """Calcula a probabilidade de transitar de state para next_state dado action"""
        if action not in self.dice:
            raise ValueError("Invalid action")
        
        current_index = self.states.index(state)
        next_index = self.states.index(next_state)
        
        count = 0
        for roll in self.dice[action]:
            if (current_index + roll) % len(self.states) == next_index:
                count += 1
        
        probability = count / len(self.dice[action])
        return probability

# Uso do ambiente
env = CafeAcarajeGame()
state = env.reset()
print(f"Estado inicial: {state}")
num_steps = 0

for _ in range(10):
    action = np.random.choice(["D1", "D2", "D3"])  # Escolher um dado aleatoriamente
    next_state, reward, done = env.step(action)
    num_steps += 1
    print(f"Action: {action}, Next state: {next_state}, Reward: {reward}, Done: {done}")
    if done:
        print("Chegou ao Café! Fim do jogo.")
        break

print(f"Número total de jogadas: {num_steps}")

# Teste da função de transição
state = "F"
action = "D1"
next_state = "D"
prob = env.transition_probability(state, action, next_state)
print(f"Probabilidade de transitar de {state} para {next_state} com ação {action}: {prob}")

state = "B"
action = "D2"
next_state = "F"
prob = env.transition_probability(state, action, next_state)
print(f"Probabilidade de transitar de {state} para {next_state} com ação {action}: {prob}")

Estado inicial: F
Action: D1, Next state: F, Reward: 0, Done: False
Action: D3, Next state: D, Reward: 0, Done: False
Action: D3, Next state: B, Reward: 0, Done: False
Action: D1, Next state: Café, Reward: 1, Done: True
Chegou ao Café! Fim do jogo.
Número total de jogadas: 4
Probabilidade de transitar de F para D com ação D1: 0.16666666666666666
Probabilidade de transitar de B para F com ação D2: 0.3333333333333333


In [18]:
t = np.array([[[env.transition_probability(s,a,s_) for s_ in env.states] for a in env.dice] for s in env.states])
r = np.array([[[env.reward(s) for s in env.states] for a in env.dice] for s_ in env.states])
#r = np.array([[np.sum([env.reward(s_) * env.transition_probability(s,a,s_) for s_ in env.states]) for a in env.dice] for s in env.states])
# v = np.array([0.0 for s in env.states])
v = np.zeros(len(env.states))
eps = 0.1
gamma = 0.9
pi = ["" for s in env.states]
while True:
    v_ = v.copy() #Esse é o v antigo
    for s,_ in enumerate(env.states):
        # print(np.max([
        #     (np.sum([t[s,0,s_] * (r[s,0,s_] + v_[s_] ) for s_,_ in enumerate(env.states)] ) * gamma),
        #     (np.sum([t[s,1,s_] * (r[s,1,s_] + v_[s_] ) for s_,_ in enumerate(env.states)] ) * gamma),
        #     (np.sum([t[s,2,s_] * (r[s,2,s_] + v_[s_] ) for s_,_ in enumerate(env.states)] ) * gamma)]))
        
        # print( np.max([gamma * np.sum([t[s,a,s_]* (r[s,a,s_] + v_[s_]) for s_,_ in enumerate(env.states)]) for a,_ in enumerate(env.dice)]) )
        v[s] = np.max([gamma * np.sum([t[s,a,s_]* (r[s,a,s_] + v_[s_]) for s_,_ in enumerate(env.states)]) for a,_ in enumerate(env.dice)])
        # print(v[s])
        action = np.argmax([gamma * np.sum( [t[s,a,s_]* (r[s,a,s_] + v_[s_]) for s_,_ in enumerate(env.states)]) for a,_ in enumerate(env.dice)] )
        pi[s] = [a for i,a in enumerate(env.dice)] [action] 
    #print(np.linalg.norm(v_ - v, ord=np.inf))
    if (np.linalg.norm(v_ - v, ord=np.inf) < (eps * ((1 - gamma) /(2 * gamma))) ):
        break

#Printar a politica
# v
pi

#Talvez mudar o environment e testar novamente.

['D2', 'D3', 'D3', 'D1', 'D1', 'D1']